## 핸드폰 카메라 연동 기본 코드

In [ ]:
import urllib.request
import cv2
import numpy as np

URL = "http://192.168.0.3:8080/shot.jpg"
while True:
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    img = cv2.imdecode(img_arr,-1)
    cv2.imshow('IPWebcam',img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# Release handle to the webcam
cv2.destroyAllWindows()

## image capture(smart phone cam)

In [4]:
import cv2
import datetime

now = datetime.datetime.now().strftime("%y-%m-%d_%H-%M-%S")
green = (0, 255, 0)
magenta = (255, 0, 255)
location = (0, 100)
font = cv2.FONT_ITALIC
temp = 1
 
# 영상 저장을 위한 VideoWriter 인스턴스 생성
fourcc = cv2.VideoWriter_fourcc(*'XVID')

#얼굴 인식용
face_cascade = cv2.CascadeClassifier()
face_cascade.load('haarcascade_frontalface_default.xml')

URL = "http://192.168.0.3:8080/shot.jpg"
while True:
    img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
    frame = cv2.imdecode(img_arr,-1)
    #더 이상 이미지가 없으면 종료
    #재생 다 됨
    if frame is None:
        break;

    # 얼굴인식 영상 처리
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur =  cv2.GaussianBlur(grayframe,(5,5), 0)
    faces = face_cascade.detectMultiScale(blur, 1.8, 2, 0, (50, 50))
    
    # 얼굴인식 여부 확인
    if len(faces) == 0:
        temp = 1
    else:
        temp += 1
    
    # 얼굴이 4프레임 이상 인식되면 4프레임마다 캡쳐
    if temp%4 == 0:
        print("캡쳐 : ", str(now))
        cv2.imwrite("image/capture/" + str(now) + ".png", frame) 
    
    # 얼굴인식여부를 화면상에 표시
    if temp < 4:
        cv2.putText(frame,"I can't recognize your face.", location, font, 1, green, 2)
    else:
        cv2.putText(frame,"I'm checking it now.", location, font, 1, magenta, 2)
        
    # 얼굴 인식된 이미지 화면 표시
    cv2.imshow('face_detect',frame)
    
    # q 를 누르면 종료, x 를 누르면 캡쳐
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
 
cv2.destroyAllWindows()

캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44
캡쳐 :  20-04-07_03-23-44


## face recognition(smart phone cam) 

In [3]:
import face_recognition
import cv2
import numpy as np
import os, glob
import urllib.request

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)

def load_img(file_path):
    img_path = []
    dir_path = os.path.join(os.getcwd(), file_path)
    folder_path = glob.glob(os.path.join(dir_path, '*'))
    for _ in folder_path:
        img_path.append(glob.glob(os.path.join(_, '*')))
    return img_path

def face_names(file_path):
    face_names = []
    dir_path = os.path.join(os.getcwd(), file_path)
    img_path = glob.glob(os.path.join(dir_path, '*'))
    for _ in img_path:
        face_names.append(_.split('\\')[-1])
    return face_names

def face_encoding(img_path):
    # Load a sample picture and learn how to recognize it.
    temp = []
    for folder in img_path:
        for _ in folder:    
            face_image = face_recognition.load_image_file(_)
            result = face_recognition.face_encodings(face_image)[0]
        temp.append(result)
    return temp

known_face_encodings = face_encoding(load_img('image/face_recognition/train_data'))
known_face_names = face_names('image/face_recognition/train_data')

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

URL = "http://192.168.0.49:8080/shot.jpg"
while True:
    try:
        img_arr = np.array(bytearray(urllib.request.urlopen(URL).read()),dtype=np.uint8)
        frame = cv2.imdecode(img_arr,-1)

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                min_val = min(face_distances)
                if matches[best_match_index]:
    #                name = known_face_names[best_match_index]
                     if min_val < 0.5:
                         name = known_face_names[best_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except:
        break

# Release handle to the webcam
cv2.destroyAllWindows()